## AI1104: Programming with AI

Homework 2: Backpropagation 

Author: Tanmay Goyal, AI20BTECH11021

In [ ]:
# importing all required modules
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def samples(N , network_type):
    '''
    generates training and testing samples, as well as training and testing labels
    
    param:
    N = total number of samples being used
    network_type = the binary operation our MLP would be learning, can be XOR, AND or OR

    returns:
    X_train = set of all training points
    X_test = set of all testing points
    y_train = set of all labels cooresponding to the training points
    y_test = set of all labels corresponding to the testing points

    '''

    # generating the covariance matrix
    cov = 0.05 * np.identity(2)
    
    # first quarter of training points would be centered around [0,0]
    mean1 = [0,0]
    x1 = np.random.multivariate_normal(mean1 , cov , N//4)
    
    # second quarter of training points would be centered around [0,1]
    mean2 = [0,1]
    x2 = np.random.multivariate_normal(mean2 , cov , N//4)

    # third quarter of training points would be centered around [1,0]
    mean3 = [1, 0]
    x3 = np.random.multivariate_normal(mean3 , cov , N//4)

    # fourth quarter of training points would be centered around [1,1]
    mean4 = [1,1]
    x4 = np.random.multivariate_normal(mean4 , cov , N//4)
    
    # collecting all these points together
    X = np.vstack((x1 , x2 , x3 , x4))
    
    # we would generate 0.8N random indices to choose our training samples
    training_samples_indices = np.random.choice([i for i in range(N)] , size = int(0.8*N) , \
                                                replace = False)
    
    # the indices not selected for training would be used for testing
    testing_samples_indices = np.array([i for i in range(N) if i not in training_samples_indices])

    # initialising X_train and X_test with a row of (0,0) which would later be deleted
    X_train = np.array([[0,0]])
    X_test = np.array([[0,0]])

    # We now partition all our points into training and testing sets, based on the indices
    for i in training_samples_indices:
       X_train = np.vstack((X_train , X[i][:]))

    for i in testing_samples_indices:
        X_test = np.vstack((X_test , X[i][:]))

    # we delete the first row of (0,0) we had used to initialise X_train and X_test
    X_train = np.delete(X_train , 0 , 0)
    X_test = np.delete(X_test , 0 , 0)

    # finally we add a column of ones alongside the datapoints
    X_train = np.hstack((np.ones((X_train.shape[0],1)) , X_train))
    X_test = np.hstack((np.ones((X_test.shape[0],1)) , X_test))  

    # based on the network type, we now prepare our set of labels, in the same sequence of 
    # means used to prepare our initial set of points

    if(network_type == "XOR"):
        y1 = np.array([0 for i in range(N//4)])  # 0 XOR 0 = 0
        y2 = np.array([1 for i in range (N//4)]) # 0 XOR 1 = 1
        y3 = np.array([1 for i in range (N//4)]) # 1 XOR 0 = 1
        y4 = np.array([0 for i in range(N//4)])  # 1 XOR 1 = 0
        Y = np.vstack((y1 , y2 , y3 , y4))
    
    elif(network_type == "AND"):
        y1 = np.array([0 for i in range(N//4)])  # 0 AND 0 = 0
        y2 = np.array([0 for i in range (N//4)]) # 0 AND 1 = 0
        y3 = np.array([0 for i in range (N//4)]) # 1 AND 0 = 0
        y4 = np.array([1 for i in range(N//4)])  # 1 AND 1 = 1
        Y = np.vstack((y1 , y2 , y3 , y4))

    elif(network_type == "OR"):
        y1 = np.array([0 for i in range(N//4)])  # 0 OR 0 = 0
        y2 = np.array([1 for i in range (N//4)]) # 0 OR 1 = 1
        y3 = np.array([1 for i in range (N//4)]) # 1 OR 0 = 1
        y4 = np.array([1 for i in range(N//4)])  # 1 OR 1 = 1
        Y = np.vstack((y1 , y2 , y3 , y4))

    # reshaping Y to N rows and 1 column to match the shape of X
    Y = Y.reshape((N,1))
    
    # initialising y_train and y_test with a row of zeroes which would later be deleted
    y_train = np.array([0])
    y_test = np.array([0])
    
    # We now partition all our points into training and testing labels, based on the indices
    for i in training_samples_indices:
       y_train = np.vstack((y_train , Y[i][:]))

    for i in testing_samples_indices:
        y_test = np.vstack((y_test , Y[i][:]))

    # we finally delete the first row of (0,0) we had used to initialise y_train and y_test
    y_train = np.delete(y_train , 0 , 0)
    y_test = np.delete(y_test , 0 , 0)

    return X_train, X_test, y_train , y_test


The non- linearity function we are using is the sigmoid, given by:

$\sigma (x) = \frac{1}{1 + e^{-x}}$

The derivative of sigmoid results in an interesting finding:

$\sigma '(x) = \frac{e^{-x}}{(1+e^{-x})^2} = \sigma(x) \times (1 - \sigma(x))$

Since, $\sigma(x)$ at any node is equal to $output$, we can write:

$\sigma '(x) = output \times (1-output)$

In [ ]:
# we use the above definitions of sigmoid to define our function

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


The update rule is as follows:

$w_{ij}'= w_{ij} - \gamma \frac{\delta R(\theta)}{\delta w_{ij}}$


We will use the following parameters:

Number of epochs = 100

Learning rate = $\gamma = 0.05$
 

In [ ]:
def backpropagate_and_report(N , network_type):
    '''
    This function will train the model, and update the weights using backpropagation.
    It will also report the training and testing error by plotting the same.

    param:
    N = total number of samples being used
    network_type = the binary operation our MLP would be learning, can be XOR, AND or OR

    returns:
    None
    
    '''

    # we would first generate the samples
    X_train , X_test , y_train , y_test = samples(N , network_type)

    # the number of epochs is equal to 100, predefined
    epochs = 100

    # the learning rate = 0.05, predefined
    learning_rate = 0.05

    # we want 20 updates per epoch, we donot want to fix the batch size because
    # then as the number of samples increase, the number of updates increase
    # i.e for lesser number of samples, updates would be lesser
    batch_size = len(X_train)//20
    
    # we make two empty lists for storing training and testing errors
    training_error_list = []
    testing_error_list = []

    # we would randomly initialise the weights between -1 and 1
    # since the datapoint is initially of the form [1 x1 x2]
    # the weights between hidden layer and input layer would look like [w0 w1 w2].T
    # for two hidden layers, we would end up with a set of weights with shape (3,2)
    # i.e 6 elements
    # for initialisation in (a,b), we use [a + (b-a) * np.random.rand()]
    weights_hidden_input = np.array([2 * np.random.rand() - 1 for i in range(6)]).reshape((3,2))
   
    # similarly the output from the hidden layer will be of the form [1 z1 z2]
    # so the weights would be of the form [w0 w1 w2].T
    # for one output layer, we would end up with a set of weights with shape (3,1)
    # i.e 3 elements
    weights_output_hidden = np.array([2 * np.random.rand() - 1 for i in range(3)]).reshape((3,1))
    
    
    # with everything set up, we can finally begin training our model 
    for e in range(epochs):
        
        # setting up our index to keep track of the batches
        for idx in range(batch_size , len(X_train) , batch_size):
           
            # we initialise weight_update matrices
            weights_update_hidden_input = np.zeros(weights_hidden_input.shape)
            weights_update_output_hidden = np.zeros(weights_output_hidden.shape)
            
            #setting up i for all the indices within that batch
            for i in range(idx - batch_size , idx):
                
                # we would pull out the data point from the training set
                x = X_train[i][:].reshape(1,3)
                
                # output from the hidden layers is equal to sigmoid(x * W)
                # we have already made sure the dimensions are compatible with one another 
                z = sigmoid(np.matmul(x , weights_hidden_input))
            
                # we insert a 1 before the z to account for the bias from the next set of weights
                z_with_1 = np.hstack((np.ones((1,1)) , z))
                   
                # the final output will be sigmoid(z * w)
                y_hat = sigmoid(np.matmul(z_with_1 , weights_output_hidden))
                    
                # error term = -2 * (y - y_hat) * sigmoid_prime
                # error term = -2 * (y - y_hat) * y_hat * (1 - y_hat)
                error_term = -2 * (y_train[i] - y_hat) * y_hat * (1-y_hat)

                # each weight between output and hidden layer is updated by 
                # learning_rate * error_term * corresponding input
                # since inputs were [1 z1 z2], we take transpose to convert them into the shape of the weights
                weights_update_output_hidden += (learning_rate * error_term * z_with_1.T)
                
                #we want the weights between ouput layer and hidden layer without the intercept term
                weights_without_bias = weights_output_hidden[1:]

                # each weight between hidden layer and input layer would be updated as follows:
                # eg. weight between first input and first hidden layer = 
                # learning rate * error term * first weight between output and hidden * z1_prime * x1
                # =  learning rate * error term * first weight between output and hidden * z1 * (1-z1) * x1
                weights_update_hidden_input += (learning_rate * error_term * weights_without_bias * z.T * (1-z.T) * x).T
                
            
            # after one batch is completed, we update the weights
            weights_output_hidden -= weights_update_output_hidden
            weights_hidden_input -= weights_update_hidden_input


        # we are done with all batches and one epoch is done, 
        # so now we calculate training error and testing error:
        training_error = 0
        testing_error = 0
        
        #calculating training error
        for idx in range(len(X_train)):
                    
            # we would pull out the data point from the training set    
            x = X_train[idx][:].reshape(1,3)
                    
            # output from the hidden layers is equal to sigmoid(x * W)
            # we have already made sure the dimensions are compatible with one another 
            z = sigmoid(np.matmul(x , weights_hidden_input))
            
            # we insert a 1 before the z to account for the bias from the next set of weights
            z_with_1 = np.hstack((np.ones((1,1)) , z))
                    
            # the final output will be sigmoid(z * w)
            y_hat = sigmoid(np.matmul(z_with_1 , weights_output_hidden))
            
            # error = (y-y_hat)^2
            training_error += (y_train[idx] - y_hat) ** 2

        #averaging the error
        training_error /= len(X_train)

        # calculating the testing error
        for idx in range(len(X_test)):
            
            # we would pull out the data point from the training set        
            x = X_test[idx][:].reshape(1,3)
                    
            # output from the hidden layers is equal to sigmoid(x * W)
            # we have already made sure the dimensions are compatible with one another 
            z = sigmoid(np.matmul(x , weights_hidden_input))
                
            # we insert a 1 before the z to account for the bias from the next set of weights
            z_with_1 = np.hstack((np.ones((1,1)) , z))
                    
            # the final output will be sigmoid(z * w)
            y_hat = sigmoid(np.matmul(z_with_1 , weights_output_hidden))
                    
            # error = (y-y_hat)^2
            testing_error += (y_test[idx] - y_hat) ** 2

        # averaging the error
        testing_error /= len(X_test)


        #appending the errors to the list
        training_error_list.append(training_error)
        testing_error_list.append(testing_error)
    
    # we are done with all epochs
    # we would resize our lists of errors to avoid 2D or 3D arrays
    training_error_list = np.array(training_error_list).reshape((epochs,1))
    testing_error_list = np.array(testing_error_list).reshape((epochs,1))
    
    # a list of indices from 1 to epochs representing the epoch number
    indices = [i for i in range(1,epochs+1)]
    
    # printing the final set of weights
    print("Final set of weights:")
    print("Weights between input layer and hidden layer : \n" , weights_hidden_input)
    print("Weights between hidden layer and output layer: \n" , weights_output_hidden)

    # plotting the training and testing errors
    plt.plot(indices , training_error_list , 'r' , label = "Training error")
    plt.plot(indices , testing_error_list , 'b' , label = "Testing error")
    plt.title("Training and Testing errors for N = {} and operation = {}".format(N , network_type))
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.grid(True)
    plt.show() 



## Results

### 1. XOR model for $N = 1000$

In [ ]:
backpropagate_and_report(1000 , "XOR")


### 1. XOR model for $N = 2500$

In [ ]:
backpropagate_and_report(2500 , "XOR")

### 1. XOR Model for $N = 5000$

In [ ]:
backpropagate_and_report(5000 , "XOR")

### 2. AND model for $N = 1000$

In [ ]:
backpropagate_and_report(1000 , "AND")

### 2. AND Model for $N = 2500$

In [ ]:
backpropagate_and_report(2500 , "AND")

### 2. AND model for $N = 5000$

In [ ]:
backpropagate_and_report(5000 , "AND")

### 3. OR Model for $N = 1000$

In [ ]:
backpropagate_and_report(1000 , "OR")

### 3. OR model for N = 2500

In [ ]:
backpropagate_and_report(2500 , "OR")

### 3. OR model for $N = 5000$

In [ ]:
backpropagate_and_report(5000 , "OR")

### General observations
1. In general, as N increases, the training and testing errors tends to decrease.
<br>
<br>
2. Due to the random initialisation of weights, sometimes we get the testing error to be greater than training error, and sometimes vice- versa. Thus, most of the observations have been made keeping in mind the majority of those runs.
<br>
<br>
3. None of the models are underfitting or overfitting. In fact, in most of the models, the training and testing errors turn out to pretty similar. During some runs, the training error might marginally exceed the testing error, or vice- versa, but in general, they are close to each other, and hence, we can say the models are decently trained.
<br>
<br>
<b><u>Note: </u></b> the above observations have been made after checking majority of the runs. One or two particular runs <b>may</b> contradict these observations. 
